# Cargue de librerias

In [74]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import re
from unicodedata import normalize
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:,.2f}'.format
import pyxlsb
from pyxlsb import convert_date

# Cargue de Datos

In [75]:
base = pd.read_excel('anexo-area-proyecciones-poblacion-Municipal_Area_1985-2017.xlsx', header=11)
base2 = pd.read_excel('anexo-proyecciones-poblacion-Municipal_Area_2018-2035.xlsx', header=11)

# Parámetros

In [76]:
#periodo de tiempo para filtrar la base
periodo = [2010,2020]

### Análisis descriptivo de los datos

In [77]:
#revisamos la estructura de la data
base.head()
base2.head()

,DP,DPNOM,COD_MUNICIPIO,MPNOM,AÑO,ÁREA GEOGRÁFICA,Total
0,05,Antioquia,"5,001.00",Medellín,"1,985.00",Cabecera Municipal,1393864
1,05,Antioquia,"5,001.00",Medellín,"1,985.00",Centros Poblados y Rural Disperso,56313
2,05,Antioquia,"5,001.00",Medellín,"1,985.00",Total,1450177
3,05,Antioquia,"5,001.00",Medellín,"1,986.00",Cabecera Municipal,1421745
4,05,Antioquia,"5,001.00",Medellín,"1,986.00",Centros Poblados y Rural Disperso,56223


,DP,DPNOM,COD_MUNICIPIO,MPNOM,AÑO,ÁREA GEOGRÁFICA,Total
0,5,Antioquia,5001,Medellín,2018,Cabecera Municipal,2382399
1,5,Antioquia,5001,Medellín,2018,Centros Poblados y Rural Disperso,44730
2,5,Antioquia,5001,Medellín,2018,Total,2427129
3,5,Antioquia,5001,Medellín,2019,Cabecera Municipal,2439470
4,5,Antioquia,5001,Medellín,2019,Centros Poblados y Rural Disperso,44075


In [78]:
#Variables identificadas
base.columns
base2.columns

Index(['DP', 'DPNOM', 'COD_MUNICIPIO', 'MPNOM', 'AÑO', 'ÁREA GEOGRÁFICA',
       'Total'],
      dtype='object')

Index(['DP', 'DPNOM', 'COD_MUNICIPIO', 'MPNOM', 'AÑO', 'ÁREA GEOGRÁFICA',
       'Total'],
      dtype='object')

In [79]:
#conteo de datos pertidos (nulos)
base.isnull().sum()
base2.isnull().sum()

DP                 0
DPNOM              1
COD_MUNICIPIO      1
MPNOM              1
AÑO                1
ÁREA GEOGRÁFICA    1
Total              0
dtype: int64

DP                 0
DPNOM              0
COD_MUNICIPIO      0
MPNOM              0
AÑO                0
ÁREA GEOGRÁFICA    0
Total              0
dtype: int64

In [80]:
#tipos de datos recibidos
base.dtypes
base2.dtypes

DP                  object
DPNOM               object
COD_MUNICIPIO      float64
MPNOM               object
AÑO                float64
ÁREA GEOGRÁFICA     object
Total                int64
dtype: object

DP                  int64
DPNOM              object
COD_MUNICIPIO       int64
MPNOM              object
AÑO                 int64
ÁREA GEOGRÁFICA    object
Total               int64
dtype: object

In [81]:
#dimensiones de la base
base.shape
base2.shape

(111079, 7)

(60588, 7)

### Estandarización de los datos

In [82]:
#se renombran las variables
base = base.rename(columns={'DP':'COD_DEP', 'DPNOM':'DEPARTAMENTO', 'COD_MUNICIPIO':'COD_MUN', 
                            'MPNOM':'MUNICIPIO', 'AÑO':'AÑO', 'ÁREA GEOGRÁFICA':'AREA_GEOGRAFICA',
       'Total':'TOTAL'})

base2 = base2.rename(columns={'DP':'COD_DEP', 'DPNOM':'DEPARTAMENTO', 'COD_MUNICIPIO':'COD_MUN', 
                            'MPNOM':'MUNICIPIO', 'AÑO':'AÑO', 'ÁREA GEOGRÁFICA':'AREA_GEOGRAFICA',
       'Total':'TOTAL'})

In [83]:
#se eliminan las filas con totales
base = base[base.COD_DEP != 'Total general']
base2 = base2[~base2.AREA_GEOGRAFICA.str.contains('Total')]

base = base[base.COD_DEP != 'Total general']
base2 = base2[~base2.AREA_GEOGRAFICA.str.contains('Total')]

In [84]:
#se estandariza la variable DEPARTAMENTO
base['DEPARTAMENTO'] = [re.sub(
                r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
                normalize( "NFD", x),  0, re.I) for x in base['DEPARTAMENTO']]
base['DEPARTAMENTO'] = base['DEPARTAMENTO'].str.upper()

base2['DEPARTAMENTO'] = [re.sub(
                r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
                normalize( "NFD", x),  0, re.I) for x in base2['DEPARTAMENTO']]
base2['DEPARTAMENTO'] = base2['DEPARTAMENTO'].str.upper()

In [85]:
#se estandariza la variable COD_MUN
base['COD_MUN'] = base.COD_MUN.astype(int)
base2['COD_MUN'] = base2.COD_MUN.astype(int)

In [86]:
#se estandariza la variable MUNICIPIO
base['MUNICIPIO'] = [re.sub(
                r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
                normalize( "NFD", x),  0, re.I) for x in base['MUNICIPIO']]
base['MUNICIPIO'] = base['MUNICIPIO'].str.upper()


base2['MUNICIPIO'] = [re.sub(
                r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
                normalize( "NFD", x),  0, re.I) for x in base2['MUNICIPIO']]
base2['MUNICIPIO'] = base2['MUNICIPIO'].str.upper()

In [87]:
#se estandariza la variable AÑO
base['AÑO'] = base.AÑO.astype(int)

base2['AÑO'] = base2.AÑO.astype(int)

In [88]:
#Se elimina la variable AREA_GEOGRAFICA
base = base.drop('AREA_GEOGRAFICA', axis=1)

base2 = base2.drop('AREA_GEOGRAFICA', axis=1)

In [89]:
base = base.groupby(['COD_DEP','DEPARTAMENTO','COD_MUN','MUNICIPIO','AÑO',]).agg(TOTAL = ('TOTAL','sum')).reset_index()

base2 = base2.groupby(['COD_DEP','DEPARTAMENTO','COD_MUN','MUNICIPIO','AÑO',]).agg(TOTAL = ('TOTAL','sum')).reset_index()

In [90]:
#se unen las dos bases en una sola
base = base.append(base2)

In [91]:
#se filtra la base
base = base[(base.AÑO >= periodo[0]) & (base.AÑO <= periodo[1]) ]

In [92]:
#previsualización de la base antes de exportar
base.head()

,COD_DEP,DEPARTAMENTO,COD_MUN,MUNICIPIO,AÑO,TOTAL
25,05,ANTIOQUIA,5001,MEDELLIN,2010,4371078
26,05,ANTIOQUIA,5001,MEDELLIN,2011,4427098
27,05,ANTIOQUIA,5001,MEDELLIN,2012,4481380
28,05,ANTIOQUIA,5001,MEDELLIN,2013,4533938
29,05,ANTIOQUIA,5001,MEDELLIN,2014,4587202


# Exportar Datos

In [93]:
base.to_csv('base_dane_visualizacion.csv', sep = ',', index=False)